<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2024/blob/main/IMT_ex_combine_reshape_join_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# Lab: Reshape, Combinação e Joins

Referências e Materiais úteis para este Lab:

- [Notas de Aula](https://colab.research.google.com/github/Rogerio-mack/IMT_Ciencia_de_Dados/blob/main/IMT_Pandas_I.ipynb)
- [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html#user-guide)
- [Pandas Reshaping](https://pandas.pydata.org/docs/user_guide/reshaping.html)
---



In [ ]:
import numpy as np
import pandas as pd


# **CASE: Yahoo Finance**

Nos exercícios a seguir partimos da seguinte base de dados extraída do `yahoo finance`:

In [ ]:
df = pd.read_csv('https://github.com/Rogerio-mack/IMT_CD_2024/raw/main/data/yahoo_finance_2019_2023.csv')
df.head()

,Date,BRL=X,BTC-USD,ITUB4.SA,PETR4.SA,^BVSP
0,2019-01-01,3.8800,3843.520020,NaN,NaN,NaN
1,2019-01-02,3.8799,3943.409424,29.194490,9.149798,91012.0
2,2019-01-03,3.7863,3836.741211,29.675810,9.374168,91564.0
3,2019-01-04,3.7551,3857.717529,29.178703,9.400789,91841.0
4,2019-01-05,NaN,3845.194580,NaN,NaN,NaN


# Q1.

Empregue o método `melt()` e a função `pd.to_datetime(df.Date).dt.day_name()` para transformar o dataframe `df` para o formato abaixo de `df_melt`.


<img src="https://pandas.pydata.org/docs/_images/reshaping_melt.png" width=800, align="right">



In [ ]:
df_melt = df.melt(id_vars='Date', value_vars=['BRL=X', 'ITUB4.SA', 'PETR4.SA', '^BVSP', 'BTC-USD'], var_name='Ticket', value_name='Adj Close' )
df_melt.head()


,Date,Ticket,Adj Close
0,2019-01-01,BRL=X,3.8800
1,2019-01-02,BRL=X,3.8799
2,2019-01-03,BRL=X,3.7863
3,2019-01-04,BRL=X,3.7551
4,2019-01-05,BRL=X,NaN


In [ ]:
from datetime import datetime

df_melt['day_of_week'] = pd.to_datetime(df_melt.Date).dt.day_name()
df_melt.head()


,Date,Ticket,Adj Close,day_of_week
0,2019-01-01,BRL=X,3.8800,Tuesday
1,2019-01-02,BRL=X,3.8799,Wednesday
2,2019-01-03,BRL=X,3.7863,Thursday
3,2019-01-04,BRL=X,3.7551,Friday
4,2019-01-05,BRL=X,NaN,Saturday


In [ ]:
#@markdown Must be True
len(df_melt[ df_melt.day_of_week == 'Tuesday' ]) == 1090

True

Qual o maior valor do Real às sextas-feiras (`Friday`) dentro do período e em que data isso ocorreu? (Atenção, a base contém cotações do dólar).

In [ ]:
df_melt[ (df_melt.day_of_week == 'Friday') & (df_melt.Ticket == 'BRL=X') ].nsmallest(5,'Adj Close')

,Date,Ticket,Adj Close,day_of_week
31,2019-02-01,BRL=X,3.6428,Friday
10,2019-01-11,BRL=X,3.7079,Friday
38,2019-02-08,BRL=X,3.7149,Friday
199,2019-07-19,BRL=X,3.7174,Friday
45,2019-02-15,BRL=X,3.7220,Friday


## Q2.

Empregue o método `pivot()` a partir do dataframe `df_melt` criar o dataframe `df_Friday` somente com os valores de fechamento da semana (`Friday`) dos ativos no formato abaixo.

<img src="https://pandas.pydata.org/docs/_images/reshaping_pivot.png" width=800, align="right">

In [ ]:
df_Friday = df_melt[ df_melt.day_of_week == 'Friday' ]
df_Friday = df_Friday.pivot(index='Date', columns='Ticket', values='Adj Close')
df_Friday.head()



Ticket,BRL=X,BTC-USD,ITUB4.SA,PETR4.SA,^BVSP
Date,,,,,
2019-01-04,3.7551,3857.717529,29.178703,9.400789,91841.0
2019-01-11,3.7079,3687.365479,29.667917,9.503470,93658.0
2019-01-18,3.7489,3657.839355,29.715265,9.659387,96097.0
2019-01-25,3.7698,3599.765869,NaN,NaN,NaN
2019-02-01,3.6428,3487.945312,30.634508,9.811504,97861.0


In [ ]:
#@markdown Must be True
int(df_Friday['BRL=X'].sum()) == 1067

True

Qual a média da cotação do dólar nas últimas 4 sextas-feiras da base?

In [ ]:
df_Friday.iloc[-4::]

Ticket,BRL=X,BTC-USD,ITUB4.SA,PETR4.SA,^BVSP
Date,,,,,
2023-02-03,5.0492,23449.322266,23.057737,19.154360,108490.0
2023-02-10,5.2895,21651.183594,23.681910,20.645603,108150.0
2023-02-17,5.2168,24565.601562,24.673246,20.429256,109303.0
2023-02-24,5.1362,23198.126953,23.562584,20.012018,105873.0


In [ ]:
df_Friday.iloc[-4::]['BRL=X'].mean()

5.172925114631653

## Q3.

Empregue o método `concat()` para adicionar ao dataframe original `df` os dados restantes do mês de março de 2023.

In [ ]:
import yfinance as yf

df_rest_2023 = yf.download(['^BVSP', 'PETR4.SA', 'ITUB4.SA', 'BRL=X', 'BTC-USD'],'2023-03-02','2023-03-31')
df_rest_2023.head()

[*********************100%%**********************]  5 of 5 completed


Price      Adj Close                                                 Close  \
Ticker         BRL=X       BTC-USD   ITUB4.SA   PETR4.SA     ^BVSP   BRL=X   
Date                                                                         
2023-03-02    5.1783  23475.466797  22.210325  19.038462  103326.0  5.1783   
2023-03-03    5.1996  22362.679688  22.237879  19.857485  103866.0  5.1996   
2023-03-04       NaN  22353.349609        NaN        NaN       NaN     NaN   
2023-03-05       NaN  22435.513672        NaN        NaN       NaN     NaN   
2023-03-06    5.1944  22429.757812  22.743074  20.058376  104700.0  5.1944   

Price                                                     ...    Open  \
Ticker           BTC-USD   ITUB4.SA   PETR4.SA     ^BVSP  ...   BRL=X   
Date                                                      ...           
2023-03-02  23475.466797  24.180000  24.639999  103326.0  ...  5.1783   
2023-03-03  22362.679688  24.209999  25.700001  103866.0  ...  5.1996   
2023-03-04  22353.349609        NaN        NaN       NaN  ...     NaN   
2023-03-05  22435.513672        NaN        NaN       NaN  ...     NaN   
2023-03-06  22429.757812  24.760000  25.959999  104700.0  ...  5.1944   

Price                                                    Volume               \
Ticker           BTC-USD   ITUB4.SA   PETR4.SA     ^BVSP  BRL=X      BTC-USD   
Date                                                                           
2023-03-02  23647.019531  24.900000  25.420000  104375.0    0.0  20386398516   
2023-03-03  23476.632812  24.340000  24.830000  103326.0    0.0  26062404610   
2023-03-04  22362.923828        NaN        NaN       NaN    NaN  11166012913   
2023-03-05  22354.144531        NaN        NaN       NaN    NaN  13317001733   
2023-03-06  22436.816406  24.440001  25.700001  103865.0    0.0  17353192895   

Price                                            
Ticker        ITUB4.SA     PETR4.SA       ^BVSP  
Date                                             
2023-03-02  46105200.0  124516200.0  14738800.0  
2023-03-03  25609300.0   87591800.0  13101500.0  
2023-03-04         NaN          NaN         NaN  
2023-03-05         NaN          NaN         NaN  
2023-03-06  31041400.0   50227400.0  14277800.0  

[5 rows x 30 columns]

In [ ]:
df_rest_2023 = df_rest_2023['Adj Close']

In [ ]:
df_rest_2023.head()

Ticker,BRL=X,BTC-USD,ITUB4.SA,PETR4.SA,^BVSP
Date,,,,,
2023-03-02,5.1783,23475.466797,22.210325,19.038462,103326.0
2023-03-03,5.1996,22362.679688,22.237879,19.857485,103866.0
2023-03-04,NaN,22353.349609,NaN,NaN,NaN
2023-03-05,NaN,22435.513672,NaN,NaN,NaN
2023-03-06,5.1944,22429.757812,22.743074,20.058376,104700.0


In [ ]:
df_concat = pd.concat([df,df_rest_2023])
df_concat.head()

,Date,BRL=X,BTC-USD,ITUB4.SA,PETR4.SA,^BVSP
0,2019-01-01,3.8800,3843.520020,NaN,NaN,NaN
1,2019-01-02,3.8799,3943.409424,29.194490,9.149798,91012.0
2,2019-01-03,3.7863,3836.741211,29.675810,9.374168,91564.0
3,2019-01-04,3.7551,3857.717529,29.178703,9.400789,91841.0
4,2019-01-05,NaN,3845.194580,NaN,NaN,NaN


In [ ]:
#@markdown Must be True
len(df_concat) == 1549

True

Qual o rendimento do Real em todo o período?

In [ ]:
(1/df_concat.iloc[-1,1] - 1/df_concat.iloc[0,1]) / (1/df_concat.iloc[0,1])

-0.2442834245269753

# **CASE: Gapminder**

Os dados dos exercícios a seguir tem como fonte o site [Gapmider](https://www.gapminder.org/).

<img src="https://www.gapminder.org/wp-content/themes/gapminder2/images/gapminder-logo.svg" width=500, align="center">

In [ ]:
country_entities = pd.read_csv('https://github.com/open-numbers/ddf--gapminder--co2_emission/raw/master/ddf--entities--country.csv')
country_entities.head()

,country,name
0,abkh,Abkhazia
1,abw,Aruba
2,afg,Afghanistan
3,ago,Angola
4,aia,Anguilla


In [ ]:
co2_emissions = pd.read_csv('https://github.com/open-numbers/ddf--gapminder--co2_emission/raw/master/ddf--datapoints--yearly_co2_emissions_1000_tonnes--by--country--year.csv')
co2_emissions.head()

,country,year,yearly_co2_emissions_1000_tonnes
0,abw,1959,719.95990
1,abw,1960,619.06728
2,abw,1961,646.08113
3,abw,1962,709.52273
4,abw,1963,679.64378


In [ ]:
coal_eletricity = pd.read_csv('https://github.com/open-numbers/ddf--bp--energy/raw/master/ddf--datapoints--coal_electricity_generation_twh--by--geo--year.csv')
coal_eletricity.head()

,geo,year,coal_electricity_generation_twh
0,argentina,1985,0.607
1,argentina,1986,1.013
2,argentina,1987,1.016
3,argentina,1988,1.586
4,argentina,1989,0.865


# Q4.  

Faça um 'inner join' (`merge`) dos dados de `country_entities,co2_emissions`. Excluindo o ano de 2018, quais países (3) tem a data mais recente das emissões inferior a 1990?

Dica: use o agregador `idxmax()`.


In [ ]:
df = pd.merge(country_entities,co2_emissions)
df.head()

,country,name,year,yearly_co2_emissions_1000_tonnes
0,abw,Aruba,1959,719.95990
1,abw,Aruba,1960,619.06728
2,abw,Aruba,1961,646.08113
3,abw,Aruba,1962,709.52273
4,abw,Aruba,1963,679.64378


In [ ]:
df_max = df.loc[df.groupby('name')['year'].idxmax()]
df_max[ df_max.year != 2018 ].sort_values('year')

,country,name,year,yearly_co2_emissions_1000_tonnes
14734,pri,Puerto Rico,1920,209.019
10546,korea_union,United Korea (former),1944,19218.747
4471,cxr,Christmas Island,1983,80.674
5031,deu_east,East Germany,1990,310070.519
19983,yem_south,South Yemen (former),1990,5830.530
5077,deu_west,West Germany,1990,703660.630
19942,yem_north,North Yemen (former),1990,3766.009
20080,yug,Yugoslavia,1991,94278.570
3502,cheslo,Czechoslovakia,1991,200819.588
19169,ussr,USSR,1991,3659317.635


# Q5.  

Faça um 'join' (`merge`) dos dados anteriores com `coal_eletricity`. Inclua no join dados de `coal_eletricity` mesmo que não haja o correspondente nos dados anteriores. Qual a média de `coal_electricity_generation_twh` para a linhas que não encontraram correspondentes.



In [ ]:
print( len(pd.merge(df, coal_eletricity)))
print( len(pd.merge(df, coal_eletricity, how='outer')))
print( len(pd.merge(df, coal_eletricity, how='left')))
print( len(pd.merge(df, coal_eletricity, how='right')))

326040
339057
339013
326084


In [ ]:
df_ele = pd.merge(df, coal_eletricity, how='right')
df_ele.head()

,country,name,year,yearly_co2_emissions_1000_tonnes,geo,coal_electricity_generation_twh
0,abw,aruba,1985,937.29856,argentina,0.607
1,afg,afghanistan,1985,3504.75616,argentina,0.607
2,ago,angola,1985,4679.94294,argentina,0.607
3,alb,albania,1985,7880.38300,argentina,0.607
4,are,united arab emirates,1985,49693.18922,argentina,0.607


In [ ]:
#@markdown Must be True
len(df_ele) == 326084

True

In [ ]:
df_ele[ df_ele.name.isnull() ].coal_electricity_generation_twh.mean()

669.823693608409